In [1]:
from six.moves import cPickle as pickle
from collections import deque
import numpy as np

In [2]:
with open("d.pkl","rb") as f:           
    data = pickle.load(f)               
    sentences = data ["sentences"]      
    word2int = data ["word2int"]        
    int2word = data ["int2word"]      
    del data

### Batch generator

In [3]:
def sentence_to_skipgram(sentence, window_size, start_index, required_size):
    global word2int
    skipgram = deque()
    for i, target in enumerate(sentence[start_index:]):
        start = max(0, i-window_size)
        end = min(i+window_size+1, len(sentence))
        context_words = deque([word2int[w] for w in sentence[start:end] if w!= target])

        while len(context_words) > 0:
            skipgram.append((word2int[target], context_words.popleft()))

            if len(skipgram)>= required_size:
                return np.array(skipgram, dtype=np.int32), -1 if i+start_index >= len(sentence) else i+start_index
    return np.array(skipgram, dtype=np.int32), -1

In [4]:
def sentence_generator():
    i = -1
    while 1:
        if i >= len(sentences)-1:
            i = 0
        else:
            i += 1
        yield sentences[i]



def generate_batch(bs=128, w=5):
    i = 0
    start_index = -1
    while 1:
        batch = None
        size = 0
        
        if start_index == -1:  # all sentence has been processed
            s = sentences[i]  # get next sentence in the corpus
            i += 1
            start_index = 0

        while size < bs:
            new, start_index = sentence_to_skipgram(s, w, start_index,
                                                    bs-size)

            if batch is None:
                batch = new
            else:
                batch = np.concatenate((batch, new))
            size = len(batch)
#         y = batch[:, 1].reshape([len(x), 1])
        yield batch[:, 0], batch[:, 1]

In [5]:
g = generate_batch()
for i in range(1000):
    print(i)
    g.__next__()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134


KeyboardInterrupt: 